# GameFace
HCI With Head Pose Using Computer Vision

---
Practice Module: Intelligent Sensing System (ISS)


# 0. File Path & Library Setup

In [1]:
# Load All Necessary Packages

import os
import pathlib
# from google.colab import drive

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import math
from numpy import linalg
from scipy.spatial.transform import Rotation
import scipy.io as sio
import cv2
import time
from IPython.display import clear_output

seed = 12

print("Versions of key libraries")
print("-------------------------")
print("pandas:      ", pd.__version__)
print("numpy:       ", np.__version__)
print("opencv:      ", cv2.__version__)

Versions of key libraries
-------------------------
pandas:       1.3.3
numpy:        1.19.5
opencv:       4.5.3


In [2]:
# # Mounting to Google Drive
# drive.mount('/content/gdrive')

# # Change Working Directory
# os.chdir('/content/gdrive/My Drive/iss/prs_pm/training')

print('Working Directory: ')
# !pwd
os.getcwd()


Working Directory: 


'D:\\Personal\\Education\\NUS-ISS Mtech IS\\Course Materials\\3. Intelligent Sensing Systems (ISS)\\0. Practice Module (TT)\\Source Code\\Face Pose Estimation 1'

# 1. Load Data File Path

## Load Data Directory

In [3]:
# Define dataset path
hp300wlpdir = 'Data/300W_LP'
hp300wlpoutdir = 'Data/300W_LP_processed'


## Data Paths

In [4]:
# Get All Image and Annotation Paths
folder = os.listdir(hp300wlpdir)

dataCount = [] # Arranged by personID (1-24)
imgPath = [] 
annPath = [] # Not used
for i in range(len(folder)):
    path = os.path.join(os.getcwd(), hp300wlpdir, folder[i])
    file_list = os.listdir(path)
    data_count = 0
    for file in file_list:
        if file.endswith('.jpg'):
            data_count += 1
            imgPath.append(os.path.join(path, file))
        if file.endswith('.mat'):
            annPath.append(os.path.join(path, file))
    dataCount.append(data_count)

print('Total Number of PersonID      : ', str(len(folder)))
print('Total Number of Images        : ', str(len(imgPath)))
print('Total Number of Annotations   : ', str(len(annPath)))
print('Number of Images per PersonID : ', dataCount)


Total Number of PersonID      :  8
Total Number of Images        :  122450
Total Number of Annotations   :  122450
Number of Images per PersonID :  [5207, 5207, 37676, 37676, 1786, 1786, 16556, 16556]


In [5]:
# Testing Loaded Paths
testing = cv2.imread(imgPath[0])
cv2.imshow('testing', testing)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 2. Extract Face and Euler Angles

In [6]:
# Utility Functions

def get_bbox(mat, scale, nrow, nclm):
    lm = mat['pt2d']
    min_clm = min(lm[0,:])
    min_row = min(lm[1,:])
    max_clm = max(lm[0,:])
    max_row = max(lm[1,:])
    height = max_row - min_row
    width = max_clm - min_clm
    new_min_row = max(min_row - (scale - 1)*height/2, 0)
    new_max_row = min(max_row + (scale - 1)*height/2, nrow)
    new_min_clm = max(min_clm - (scale - 1)*width/2, 0)
    new_max_clm = min(max_clm + (scale - 1)*width/2, nclm)
    return(int(min_row), int(min_clm), int(max_row), int(max_clm))

def get_euler_txt(mat):
    euler = mat['Pose_Para'][0][:3]
    pitch = euler[0]  * 180 / np.pi
    yaw = euler[1] * 180 / np.pi
    roll = euler[2] * 180 / np.pi
    euler_txt = ','.join([str(pitch), str(yaw), str(roll)])
    return euler_txt


In [7]:
# Create Empty Folders

for i in folder:
    folder_path = pathlib.Path(os.path.join(os.getcwd(), hp300wlpoutdir, i))
    folder_path.mkdir(parents=True, exist_ok=True) # Make new directory if not exists

In [8]:
# Extract Face and Euler and Save in output folder

scale = 1.3

for i in range(len(imgPath)):
    if i % 100 == 0:
        clear_output(wait=True)
        print('Processed image: {0:06d}/{1:d}'.format(i+1, len(imgPath))) 
    img_file = imgPath[i]
    mat_file = img_file.replace('.jpg', '.mat')
    img = cv2.imread(img_file)
    mat = sio.loadmat(mat_file)
    nrow, nclm, _ = img.shape

    # Get Cropped Bounding Box of Face
    bbox = get_bbox(mat, scale, nrow, nclm)
    face_img = img[bbox[0]:bbox[2], bbox[1]:bbox[3]]

    # Get Euler Angles
    euler = get_euler_txt(mat)

    # Save Face Image Output
    img_out_file = img_file.replace(hp300wlpdir, hp300wlpoutdir).replace('.jpg', '_rgb.jpg')
    cv2.imwrite(img_out_file, face_img)

    # Save Euler Angles in text
    eur_out_file = img_out_file.replace('_rgb.jpg', '_euler.txt')
    euler_write = open(eur_out_file, 'w')
    euler_write.write(euler)
    euler_write.close()
    
clear_output(wait=True)
print('Done!')


Done!
